In [1]:
from ortools.linear_solver import pywraplp

In [2]:
import pickle
import numpy as np

# Loading the list from the pickle file
file_path = r"D:\dist_list.pickle"  # Update the file path if necessary

with open(file_path, 'rb') as file:
    dist_list = pickle.load(file)

print("Dist list loaded successfully.")
print(np.shape(dist_list))

Dist list loaded successfully.
(400, 1000)


In [3]:
dist_list =  np.transpose(dist_list, (1, 0))

In [15]:
costs = [
    [90, 80, 75, 70],
    [35, 85, 55, 65],
    [125, 95, 90, 95],
    [45, 110, 95, 115],
    [60, 120, 50, 100],
    [50, 100, 90, 400],
    [50, 200, 170, 120],
    [50, 200, 190, 160]
]
num_workers = len(costs)
num_tasks = len(costs[0])
# Create the mip solver with the SCIP backend.
solver = pywraplp.Solver.CreateSolver("SCIP")

# x[i, j] is an array of 0-1 variables, which will be 1
# if worker i is assigned to task j.
x = {}
for i in range(num_workers):
    for j in range(num_tasks):
        x[i, j] = solver.IntVar(0, 1, "")
        
# Each worker is assigned to at most 1 task.
for i in range(num_workers):
    solver.Add(solver.Sum([x[i, j] for j in range(num_tasks)]) <= 1)

# Each task is assigned to exactly one worker.
for j in range(num_tasks):
    solver.Add(solver.Sum([x[i, j] for i in range(num_workers)]) == 1)
    
objective_terms = []
for i in range(num_workers):
    for j in range(num_tasks):
        objective_terms.append(costs[i][j] * x[i, j])
solver.Minimize(solver.Sum(objective_terms))
status = solver.Solve()
if status == pywraplp.Solver.OPTIMAL or status == pywraplp.Solver.FEASIBLE:
    print(f"Total cost = {solver.Objective().Value()}\n")
    for i in range(num_workers):
        for j in range(num_tasks):
            # Test if x[i,j] is 1 (with tolerance for floating point arithmetic).
            if x[i, j].solution_value() > 0.5:
                print(f"Worker {i} assigned to task {j}." + f" Cost: {costs[i][j]}")
else:
    print("No solution found.")


Total cost = 240.0

Worker 0 assigned to task 1. Cost: 80
Worker 1 assigned to task 3. Cost: 65
Worker 3 assigned to task 0. Cost: 45
Worker 4 assigned to task 2. Cost: 50


In [5]:
from tqdm import tqdm

costs = dist_list

num_workers = len(costs)
num_tasks = len(costs[0])
# Create the mip solver with the SCIP backend.
solver = pywraplp.Solver.CreateSolver("SCIP")

# x[i, j] is an array of 0-1 variables, which will be 1
# if worker i is assigned to task j.
x = {}
for i in range(num_workers):
    for j in range(num_tasks):
        x[i, j] = solver.IntVar(0, 1, "")
        
# Each worker is assigned to at most 1 task.
for i in range(num_workers):
    solver.Add(solver.Sum([x[i, j] for j in range(num_tasks)]) <= 1)

# Each task is assigned to exactly one worker.
for j in range(num_tasks):
    solver.Add(solver.Sum([x[i, j] for i in range(num_workers)]) == 1)
    
objective_terms = []
for i in tqdm(range(num_workers)):
    for j in range(num_tasks):
        objective_terms.append(costs[i][j] * x[i, j])
solver.Minimize(solver.Sum(objective_terms))

status = solver.Solve()

sol_indexes = []
if status == pywraplp.Solver.OPTIMAL or status == pywraplp.Solver.FEASIBLE:
    print(f"Total cost = {solver.Objective().Value()}\n")
    for i in range(num_workers):
        for j in range(num_tasks):
            # Test if x[i,j] is 1 (with tolerance for floating point arithmetic).
            if x[i, j].solution_value() > 0.1:
                print(f"Cluster {j} assigned to Class {i}." + f" Cost: {costs[i][j]}")
                sol_indexes.append(i)
else:
    print("No solution found.")

100%|██████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:13<00:00, 76.78it/s]


Total cost = 1235.164815143158

Cluster 262 assigned to Class 0. Cost: 1.7466256273122005
Cluster 77 assigned to Class 1. Cost: 3.732581260347412
Cluster 16 assigned to Class 2. Cost: 1.4666491774451602
Cluster 201 assigned to Class 3. Cost: 2.522022283511766
Cluster 98 assigned to Class 4. Cost: 2.643661020522324
Cluster 164 assigned to Class 5. Cost: 2.391574634896179
Cluster 313 assigned to Class 6. Cost: 3.6768236987973704
Cluster 198 assigned to Class 10. Cost: 1.7763014109012507
Cluster 273 assigned to Class 14. Cost: 2.3966022547671515
Cluster 39 assigned to Class 18. Cost: 1.7865650781781128
Cluster 301 assigned to Class 20. Cost: 2.3634370347320877
Cluster 238 assigned to Class 21. Cost: 2.9620721804343355
Cluster 321 assigned to Class 25. Cost: 1.1640495388430776
Cluster 119 assigned to Class 27. Cost: 1.6125174903525485
Cluster 394 assigned to Class 29. Cost: 2.255991931571557
Cluster 116 assigned to Class 31. Cost: 4.158150117166887
Cluster 167 assigned to Class 33. Cost: 2

Cluster 244 assigned to Class 754. Cost: 4.008920768584003
Cluster 245 assigned to Class 755. Cost: 2.379391968492955
Cluster 78 assigned to Class 757. Cost: 4.545166560385473
Cluster 359 assigned to Class 759. Cost: 3.8900289474010576
Cluster 136 assigned to Class 760. Cost: 2.1362070818709094
Cluster 106 assigned to Class 764. Cost: 2.798791686513564
Cluster 107 assigned to Class 779. Cost: 7.861437864163278
Cluster 89 assigned to Class 780. Cost: 2.783836226508379
Cluster 314 assigned to Class 782. Cost: 4.518169549733125
Cluster 371 assigned to Class 785. Cost: 11.8369218493971
Cluster 8 assigned to Class 789. Cost: 3.6680796254064623
Cluster 21 assigned to Class 791. Cost: 4.265963506804536
Cluster 399 assigned to Class 794. Cost: 1.2436848010434618
Cluster 380 assigned to Class 796. Cost: 2.7652584395042656
Cluster 131 assigned to Class 798. Cost: 1.3833325326524772
Cluster 213 assigned to Class 800. Cost: 1.877162299419865
Cluster 75 assigned to Class 802. Cost: 3.37920363725451

In [6]:
import pickle

# Saving the list to a file in the D: drive
file_path = r"D:\index_list.pickle"  # r prefix is used for raw string to avoid escape characters

with open(file_path, 'wb') as file:
    pickle.dump(sol_indexes, file)

print("Combo list saved successfully.")


Combo list saved successfully.
